In [2]:
import boto3
import pandas as pd

import seaborn as sns
from matplotlib import pyplot as plt
sns.set(rc={'figure.figsize':(24,12)})

In [4]:
df = pd.read_csv('s3://adfc-bike-census/adfc_data/transformed/combined.csv')
df['date'] = pd.to_datetime(df['date'])

In [5]:
df.head()


,city,station_code,date,count,timestamp
0,Aschaffenburg,1,2018-08-10 00:00:00,1,1533859200000
1,Aschaffenburg,1,2018-08-10 00:15:00,0,1533860100000
2,Aschaffenburg,1,2018-08-10 00:30:00,0,1533861000000
3,Aschaffenburg,1,2018-08-10 00:45:00,0,1533861900000
4,Aschaffenburg,1,2018-08-10 01:00:00,1,1533862800000


In [6]:
df_berlin = df.loc[df['city'] == 'Berlin']

In [7]:
df_berlin['datum'] = df_berlin['date'].dt.date
df_berlin['hour'] = df_berlin['date'].dt.hour

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [8]:
df_berlin

,city,station_code,date,count,timestamp,datum,hour
110100,Berlin,1,2016-01-01 00:00:00,0,1451606400000,2016-01-01,0
110101,Berlin,1,2016-01-01 00:15:00,3,1451607300000,2016-01-01,0
110102,Berlin,1,2016-01-01 00:30:00,1,1451608200000,2016-01-01,0
110103,Berlin,1,2016-01-01 00:45:00,3,1451609100000,2016-01-01,0
110104,Berlin,1,2016-01-01 01:00:00,1,1451610000000,2016-01-01,1
...,...,...,...,...,...,...,...
3426213,Berlin,18,2021-09-29 22:45:00,28,1632955500000,2021-09-29,22
3426214,Berlin,18,2021-09-29 23:00:00,19,1632956400000,2021-09-29,23
3426215,Berlin,18,2021-09-29 23:15:00,12,1632957300000,2021-09-29,23
3426216,Berlin,18,2021-09-29 23:30:00,14,1632958200000,2021-09-29,23


In [9]:
df_berlin['station_code'] == 1


110100      True
110101      True
110102      True
110103      True
110104      True
           ...  
3426213    False
3426214    False
3426215    False
3426216    False
3426217    False
Name: station_code, Length: 3316118, dtype: bool

In [38]:
hourly_grouped = df_berlin.groupby([df_berlin['datum'], df_berlin['hour'], 'city', df_berlin['station_code']], as_index=False)['count'].sum()
hourly_grouped

#<your DataFrame>.rename(columns={'count':'Total_Numbers'})

,datum,hour,city,station_code,count
0,2016-01-01,0,Berlin,1,7
1,2016-01-01,0,Berlin,2,7
2,2016-01-01,0,Berlin,5,4
3,2016-01-01,0,Berlin,8,3
4,2016-01-01,0,Berlin,9,4
...,...,...,...,...,...
829046,2021-09-29,23,Berlin,13,5
829047,2021-09-29,23,Berlin,14,146
829048,2021-09-29,23,Berlin,15,16
829049,2021-09-29,23,Berlin,17,13


In [11]:
df_location = pd.read_csv('s3://adfc-bike-census/adfc_data/fetched_eco_bike_data.csv')
df_location

,city,number,latitude,longitude,id_list,Distance to center [km],fetched_url
0,Aschaffenburg,1,49.973150,9.150790,100046798,0.1432,#NAME?
1,Berlin,1,52.514066,13.417751,100024661,1.9827,#NAME?
2,Berlin,2,52.527180,13.372020,100032152,1.6032,#NAME?
3,Berlin,3,52.513739,13.474367,100032154,5.7977,#NAME?
4,Berlin,4,52.566810,13.412170,100032155,5.7547,#NAME?
...,...,...,...,...,...,...,...
80,Stuttgart,13,48.826000,9.214880,100063205,5.8718,#NAME?
81,Stuttgart,14,48.780270,9.177410,100063206,0.2782,#NAME?
82,Stuttgart,15,48.769761,9.172275,100063225,1.1201,#NAME?
83,Osnabrück,1,52.279965,8.055056,100050631,1.0234,#NAME?


In [12]:
df_location_Berlin = df_location.loc[df_location['city'] == 'Berlin']
df_location_Berlin

,city,number,latitude,longitude,id_list,Distance to center [km],fetched_url
1,Berlin,1,52.514066,13.417751,100024661,1.9827,#NAME?
2,Berlin,2,52.527180,13.372020,100032152,1.6032,#NAME?
3,Berlin,3,52.513739,13.474367,100032154,5.7977,#NAME?
4,Berlin,4,52.566810,13.412170,100032155,5.7547,#NAME?
5,Berlin,5,52.548840,13.400370,100032159,3.6211,#NAME?
6,Berlin,6,52.533665,13.198963,100032161,12.9793,#NAME?
7,Berlin,7,52.466760,13.309160,100032163,7.7698,#NAME?
8,Berlin,8,52.488120,13.369790,100032165,3.4648,#NAME?
9,Berlin,9,52.558190,13.364940,100032169,4.8536,#NAME?
10,Berlin,10,52.493118,13.429149,100032236,3.8091,#NAME?


In [13]:
df_gesamt = hourly_grouped.merge(df_location, on = "city")

In [14]:
df_gesamt.head()

,datum,hour,city,station_code,count,number,latitude,longitude,id_list,Distance to center [km],fetched_url
0,2016-01-01,0,Berlin,1,7,1,52.514066,13.417751,100024661,1.9827,#NAME?
1,2016-01-01,0,Berlin,1,7,2,52.527180,13.372020,100032152,1.6032,#NAME?
2,2016-01-01,0,Berlin,1,7,3,52.513739,13.474367,100032154,5.7977,#NAME?
3,2016-01-01,0,Berlin,1,7,4,52.566810,13.412170,100032155,5.7547,#NAME?
4,2016-01-01,0,Berlin,1,7,5,52.548840,13.400370,100032159,3.6211,#NAME?


# Fahrradunfälle

lädt Fahrradunälle auc CSV-Dateien der Stadt Berlin

In [15]:
#!pip3 install utm
import pandas as pd
import utm
import re
from pyproj import Proj

In [16]:
df = pd.read_csv("s3://adfc-bike-census/fahrradunfaelle_berlin/unfaelle_2018.csv", sep=";", encoding="ISO-8859-1")
#df19 = pd.read_csv("s3://adfc-bike-census/fahrradunfaelle_berlin/unfaelle_2019.csv", sep=";", encoding="ISO-8859-1")
#df20 = pd.read_csv("s3://adfc-bike-census/fahrradunfaelle_berlin/unfaelle_2020.csv", sep=";", encoding="ISO-8859-1")

#df = df18.append(df19).append(df20)

In [17]:
df

,OBJECTID,LAND,BEZ,LOR,STRASSE,LOR_ab_2021,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,...,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstig,STRZUSTAND,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84
0,112695,11,2,2050602.0,Samariterviertel,2500729.0,2018,1,15,4,...,1,1,0,0,0,1,"803604,1562","5827580,738","13,4750178","52,51359681"
1,112705,11,12,12304314.0,Ziekowstraße/Freie Scholle,12500824.0,2018,1,11,2,...,1,0,0,0,0,0,"790639,5854","5835009,589","13,29102205","52,58725906"
2,112726,11,2,2040501.0,Barnimkiez,2400520.0,2018,1,9,3,...,1,1,0,0,0,0,"799827,9618","5828733,523","13,42057818","52,52601854"
3,112737,11,7,7020202.0,Volkspark (Rudolf-Wilde-Park),7200308.0,2018,1,17,2,...,1,1,0,0,0,0,"795223,509","5823525,402","13,34828776","52,48184447"
4,112747,11,3,3020209.0,Niederschönhausen,3200206.0,2018,1,15,4,...,0,1,0,0,0,1,"798261,3849","5835047,26","13,40322797","52,58347154"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13647,208846,11,4,4020311.0,Reichsstraße,4200310.0,2018,12,10,6,...,1,0,0,0,0,0,"789485,0143","5826798,422","13,26686844","52,51426051"
13648,208848,11,3,3051017.0,Rennbahnstraße,3501037.0,2018,12,10,6,...,1,0,0,0,0,1,"802289,3817","5833109,289","13,46073187","52,56387893"
13649,208850,11,2,2050601.0,Hausburgviertel,2500726.0,2018,12,9,6,...,1,0,0,0,0,1,"801882,5599","5829015,474","13,45102757","52,52741195"
13650,208851,11,11,11030618.0,Fennpfuhl West,11300618.0,2018,12,9,6,...,0,0,0,0,0,1,"802589,0571","5828801,391","13,46121525","52,52510148"


In [18]:
df.LINREFX = df.LINREFX.map(lambda x: re.sub(',', '.', x))
df.LINREFY = df.LINREFY.map(lambda x: re.sub(',', '.', x))

df.LINREFY = df.LINREFY.astype(float)
df.LINREFX = df.LINREFX.astype(float)

In [19]:
df

,OBJECTID,LAND,BEZ,LOR,STRASSE,LOR_ab_2021,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,...,IstPKW,IstFuss,IstKrad,IstGkfz,IstSonstig,STRZUSTAND,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84
0,112695,11,2,2050602.0,Samariterviertel,2500729.0,2018,1,15,4,...,1,1,0,0,0,1,803604.1562,5827580.738,"13,4750178","52,51359681"
1,112705,11,12,12304314.0,Ziekowstraße/Freie Scholle,12500824.0,2018,1,11,2,...,1,0,0,0,0,0,790639.5854,5835009.589,"13,29102205","52,58725906"
2,112726,11,2,2040501.0,Barnimkiez,2400520.0,2018,1,9,3,...,1,1,0,0,0,0,799827.9618,5828733.523,"13,42057818","52,52601854"
3,112737,11,7,7020202.0,Volkspark (Rudolf-Wilde-Park),7200308.0,2018,1,17,2,...,1,1,0,0,0,0,795223.5090,5823525.402,"13,34828776","52,48184447"
4,112747,11,3,3020209.0,Niederschönhausen,3200206.0,2018,1,15,4,...,0,1,0,0,0,1,798261.3849,5835047.260,"13,40322797","52,58347154"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13647,208846,11,4,4020311.0,Reichsstraße,4200310.0,2018,12,10,6,...,1,0,0,0,0,0,789485.0143,5826798.422,"13,26686844","52,51426051"
13648,208848,11,3,3051017.0,Rennbahnstraße,3501037.0,2018,12,10,6,...,1,0,0,0,0,1,802289.3817,5833109.289,"13,46073187","52,56387893"
13649,208850,11,2,2050601.0,Hausburgviertel,2500726.0,2018,12,9,6,...,1,0,0,0,0,1,801882.5599,5829015.474,"13,45102757","52,52741195"
13650,208851,11,11,11030618.0,Fennpfuhl West,11300618.0,2018,12,9,6,...,0,0,0,0,0,1,802589.0571,5828801.391,"13,46121525","52,52510148"


In [20]:
p = Proj(proj='utm', zone=32, ellps='WGS84')
df["latlon"] = df.apply(lambda x: p(x["LINREFX"], x["LINREFY"], inverse=True), axis = 1)

In [21]:
df_rad = df[df["IstRad"] == 1]
df_rad.head()

,OBJECTID,LAND,BEZ,LOR,STRASSE,LOR_ab_2021,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,...,IstFuss,IstKrad,IstGkfz,IstSonstig,STRZUSTAND,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84,latlon
4,112747,11,3,3020209.0,Niederschönhausen,3200206.0,2018,1,15,4,...,1,0,0,0,1,798261.3849,5835047.260,"13,40322797","52,58347154","(13.40322796953067, 52.58347153650185)"
12,112892,11,3,3071639.0,Bötzowstraße,3701660.0,2018,1,11,5,...,0,0,0,0,1,800551.7210,5829808.492,"13,43218628","52,53525526","(13.432186281821432, 52.53525525766559)"
13,112902,11,2,2050602.0,Samariterviertel,2500729.0,2018,1,8,2,...,0,0,0,0,0,803320.7292,5827627.446,"13,47089659","52,51417313","(13.470896587655746, 52.51417312732597)"
14,112921,11,1,1011204.0,Leipziger Straße ...,1100207.0,2018,1,19,4,...,0,0,0,0,1,798174.6913,5826939.978,"13,39467345","52,51084845","(13.394673449576796, 52.51084845664613)"
16,112947,11,9,9020601.0,Schnellerstraße ...,9200613.0,2018,1,18,4,...,0,0,0,0,1,806109.6297,5821644.404,"13,50637217","52,45899296","(13.5063721665925, 52.45899296616116)"


In [22]:
df_rad[df_rad["UJAHR"] ==2018]

,OBJECTID,LAND,BEZ,LOR,STRASSE,LOR_ab_2021,UJAHR,UMONAT,USTUNDE,UWOCHENTAG,...,IstFuss,IstKrad,IstGkfz,IstSonstig,STRZUSTAND,LINREFX,LINREFY,XGCSWGS84,YGCSWGS84,latlon
4,112747,11,3,3020209.0,Niederschönhausen,3200206.0,2018,1,15,4,...,1,0,0,0,1,798261.3849,5835047.260,"13,40322797","52,58347154","(13.40322796953067, 52.58347153650185)"
12,112892,11,3,3071639.0,Bötzowstraße,3701660.0,2018,1,11,5,...,0,0,0,0,1,800551.7210,5829808.492,"13,43218628","52,53525526","(13.432186281821432, 52.53525525766559)"
13,112902,11,2,2050602.0,Samariterviertel,2500729.0,2018,1,8,2,...,0,0,0,0,0,803320.7292,5827627.446,"13,47089659","52,51417313","(13.470896587655746, 52.51417312732597)"
14,112921,11,1,1011204.0,Leipziger Straße ...,1100207.0,2018,1,19,4,...,0,0,0,0,1,798174.6913,5826939.978,"13,39467345","52,51084845","(13.394673449576796, 52.51084845664613)"
16,112947,11,9,9020601.0,Schnellerstraße ...,9200613.0,2018,1,18,4,...,0,0,0,0,1,806109.6297,5821644.404,"13,50637217","52,45899296","(13.5063721665925, 52.45899296616116)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13635,208823,11,8,8010212.0,Rollberg,8100207.0,2018,12,19,7,...,0,0,0,0,0,801299.6084,5823075.973,"13,43708675","52,47449756","(13.437086752163351, 52.474497557791786)"
13636,208825,11,11,11040926.0,Weitlingstraße,11400930.0,2018,12,10,7,...,0,0,0,0,1,805511.1745,5826402.140,"13,50195094","52,50196897","(13.501950936107862, 52.501968969784954)"
13637,208826,11,4,4030621.0,Spreestadt,4300620.0,2018,12,10,7,...,0,0,0,0,1,793133.2429,5827751.134,"13,32131586","52,52085271","(13.32131585647211, 52.520852711685855)"
13648,208848,11,3,3051017.0,Rennbahnstraße,3501037.0,2018,12,10,6,...,0,0,0,0,1,802289.3817,5833109.289,"13,46073187","52,56387893","(13.460731872323066, 52.56387892908122)"


In [23]:
df_accidents = df_rad.groupby(["UJAHR", "UMONAT", "UWOCHENTAG", "USTUNDE", "latlon"], as_index=False).size()
df_accidents.head()

,UJAHR,UMONAT,UWOCHENTAG,USTUNDE,latlon,size
0,2018,1,1,13,"(13.253224995887086, 52.445928149352355)",1
1,2018,1,1,14,"(13.41193910223544, 52.531282229269756)",1
2,2018,1,1,17,"(13.455604233979928, 52.43610101156052)",1
3,2018,1,1,17,"(13.478171429150134, 52.514969983178915)",1
4,2018,1,1,18,"(13.325952614976552, 52.49443342932684)",1


In [24]:
# LatLon Tupel splitten

df_accidents["longitude"] = df_accidents.apply(lambda x: x["latlon"][0], axis=1)
df_accidents["latitude"] = df_accidents.apply(lambda x: x["latlon"][1], axis=1)

In [25]:
df_accidents.to_csv("s3://adfc-bike-census/fahrradunfaelle_berlin/cleaned.csv", sep=",", index=False)

In [32]:
df_accidents

,UJAHR,UMONAT,UWOCHENTAG,USTUNDE,latlon,size,longitude,latitude,lat_adfc,lon_adfc,distance
0,2018,1,1,13,"(13.253224995887086, 52.445928149352355)",1,13.253225,52.445928,NaN,NaN,50
1,2018,1,1,14,"(13.41193910223544, 52.531282229269756)",1,13.411939,52.531282,NaN,NaN,50
2,2018,1,1,17,"(13.455604233979928, 52.43610101156052)",1,13.455604,52.436101,NaN,NaN,50
3,2018,1,1,17,"(13.478171429150134, 52.514969983178915)",1,13.478171,52.514970,NaN,NaN,50
4,2018,1,1,18,"(13.325952614976552, 52.49443342932684)",1,13.325953,52.494433,NaN,NaN,50
...,...,...,...,...,...,...,...,...,...,...,...
5187,2018,12,7,19,"(13.437086752163351, 52.474497557791786)",1,13.437087,52.474498,NaN,NaN,50
5188,2018,12,7,19,"(13.438986252262747, 52.541081188661465)",1,13.438986,52.541081,NaN,NaN,50
5189,2018,12,7,20,"(13.453524616321932, 52.51041762186408)",1,13.453525,52.510418,NaN,NaN,50
5190,2018,12,7,20,"(13.492419969190417, 52.4913399864065)",1,13.492420,52.491340,NaN,NaN,50


# Distanz der Station zu Fahrradunfall

In [33]:
from geopy import distance
import numpy as np

df_distance = df_accidents

def iter_func(x, y):
    
    df = df_location_Berlin
    lat_adfc = 0
    lon_adfc = 0
    d_min = 50
    
    for row in df.iterrows():
        coordinate1 = (row[1][2],row[1][3])
        coordinate2 = (x,y)
        
        d = distance.geodesic(coordinate1, coordinate2).km
        if(d < d_min):
            d_min = d
        if(d_min <= 2):
            lat_adfc = row[1][2]
            lon_adfc = row[1][3]
        else:
            lat_adfc = np.nan
            lon_adfc = np.nan
    return lat_adfc, lon_adfc, d_min



#lat_adfc, lon_adfc = iter_func(52.445928,13.253225)
lat_adfc = []
lon_adfc = []
d_min = []

for row in df_accidents.iterrows():
    lat, lon, d = iter_func(row[1][6], row[1][7])
    lat_adfc.append(lat)
    lon_adfc.append(lon)
    d_min.append(d)

#lat_adfc = df_accidents.apply(lambda x: iter_func(x["latitude"], x["longitude"]), axis=1)


In [34]:
df_accidents["lat_adfc"] = lat_adfc
df_accidents["lon_adfc"] = lon_adfc
df_accidents["distance"] = d_min

In [35]:
df_accidents

,UJAHR,UMONAT,UWOCHENTAG,USTUNDE,latlon,size,longitude,latitude,lat_adfc,lon_adfc,distance
0,2018,1,1,13,"(13.253224995887086, 52.445928149352355)",1,13.253225,52.445928,NaN,NaN,50
1,2018,1,1,14,"(13.41193910223544, 52.531282229269756)",1,13.411939,52.531282,NaN,NaN,50
2,2018,1,1,17,"(13.455604233979928, 52.43610101156052)",1,13.455604,52.436101,NaN,NaN,50
3,2018,1,1,17,"(13.478171429150134, 52.514969983178915)",1,13.478171,52.514970,NaN,NaN,50
4,2018,1,1,18,"(13.325952614976552, 52.49443342932684)",1,13.325953,52.494433,NaN,NaN,50
...,...,...,...,...,...,...,...,...,...,...,...
5187,2018,12,7,19,"(13.437086752163351, 52.474497557791786)",1,13.437087,52.474498,NaN,NaN,50
5188,2018,12,7,19,"(13.438986252262747, 52.541081188661465)",1,13.438986,52.541081,NaN,NaN,50
5189,2018,12,7,20,"(13.453524616321932, 52.51041762186408)",1,13.453525,52.510418,NaN,NaN,50
5190,2018,12,7,20,"(13.492419969190417, 52.4913399864065)",1,13.492420,52.491340,NaN,NaN,50


In [36]:
df_accidents_filtered = df_accidents[df_accidents['lat_adfc'].notna()]
df_accidents_filtered

,UJAHR,UMONAT,UWOCHENTAG,USTUNDE,latlon,size,longitude,latitude,lat_adfc,lon_adfc,distance


In [37]:
df_accidents_filtered.to_csv("s3://adfc-bike-census/fahrradunfaelle_berlin/accidents_adfcstation.csv", index=False, sep=",")